# AHLT - Second delivery - Task 9.2 DDI
**Albert Rial**   
**Karen Lliguin**   

This delivery consists of solving the task 9.2 of the SemEval-2013 challenge. The task concerns classifying drug-drug interactions between pairs of drugs. 

The dataset provided contains XML files with sentences with drugs entities appearing on it and the corresponding interaction. There are four general types: mechanism, int, effect and advise. The data is already splitted in three subsets: Train, Devel and Test.

To complete this task different methods and resources have been used among them, the Stanford CoreNLP dependency parser. We have divide the task in different subgoals.

## Goal 1: Rule-based DDI
### Introduction

First, simple heurístic rules have been used to carry out the task. In this version only the information given by the Train dataset is used. The final goal is to achieve an overall F1 score of at least 0.15 on the Devel dataset.

### Data exploration


With the purpose of building significant rules (and features for the next goal), first we have done a data exploration over the Train dataset. The following aspects have been analysed for each type of interaction:
- The most common words that appear in between the two drugs that interact (clue_words).
- The most common words that appear in each sentence containing drugs that interact (sentence_words).
- The most common lemmas in which entity1 is under in the dependency tree (e1_under), its relation and the POS tag.
- The most common lemmas in which entity2 is under in the dependency tree (e2_under), its relation and the POS tag.
- Number of times entity1 is under entity2 and vice versa.
- Number of times entity1 and entity2 share the same parent and how many times is a verb.

Besides this analysis, in order to have a more clear understading of each metric regarding on how one word is seen by all the different types, we have stored the previous information so that we could search a word regarding any of the previous metrics and the information of how many times it appears for each type is shown. For intance, given the word "response" and the metric "e1_under" the following information is obtained:
```
--------------------
effect
meet_condition: 81
total: 1525
--------------------
mechanism
meet_condition: 0
total: 1118
--------------------
int
meet_condition: 0
total: 186
--------------------
advise
meet_condition: 0
total: 707
--------------------
none
meet_condition: 14
total: 21553
```
This inside information is used to build the rules.

### Details

The *analyse* function recives a sentence text and using CoreNLP it obtains the dependency graph, cleans it keeping only the relevant information and it adds the start/end to each token. Since Standford CoreNLP changes some characters (for instance parenthesis) to a string value. To deal with this we have implemented the function *handle_special_symbols* that before adding the offset it changes it back to the real character.

In [ ]:
def handle_special_symbols(node):
    if node['word'] == '-LRB-':
        node['word'] = '('
        node['lemma'] = '('
    elif node['word'] == '-RRB-':
        node['word'] = ')'
        node['lemma'] = ')'
    elif node['word'] == '-LSB-':
        node['word'] = '['
        node['lemma'] = '['
    elif node['word'] == '-RSB-':
        node['word'] = ']'
        node['lemma'] = ']'
    elif node['word'] in ["``", "''"]:
        node['word'] = '"'
        node['lemma'] = '"'
    return
    
def analyze(sent):
    if len(sent)<= 0:
        return None
    
    mytree, = my_parser.raw_parse(sent)
    tree = mytree.nodes
    ini_token = 0
                   
    # clean tree
    info = ['address', 'head', 'lemma', 'rel', 'word', 'tag']
    
    for k in sorted(tree.keys()):
        node = tree[k] 
        for key in list(node):
            if key not in info:
                del node[key]
        
        handle_special_symbols(node)
        
        if k != 0:
            # add offsets
            ini_token = sent.find(node['word'] ,ini_token)
            node['start'] = ini_token
            ini_token += len(node['word'])
            node['end'] = ini_token - 1
            
    return tree

#### Rules

**Discard**

As a first approach, we decided to use the "clue_words" found in data exploration but since this approach is very naive, the results obtained were not good enough so we discarded this option.

**Used**
Therefore we change the strategy and decided to use the relation "entityA under entityB". With this approach and using the information of the data exploration we explored rules that check the lemma in which entity1/entity2 is under in the dependency tree. After trying different lemmas for each type of interacction, the final lemmas selected for each type interaction are the following:

- Effect: response,diminish and enhance
- Int: interact and interaction
- Mechanism: absorption, metabolism and presence
- Advise: take, adjustment, avoid, recommend and contraindicate

In [ ]:
def check_interaction(analysis, entities, e1, e2):
    # Get entities
    entity1, entity2 = get_entity_nodes(analysis, entities, e1, e2)
    parent1, rel1 = get_entity_parent(analysis, entity1)
    parent2, rel2 = get_entity_parent(analysis, entity2)
    
    # Rules
    # Entity 1 is under Entity 2
    if is_under(entity1, entity2):
        return (0, "null")
    
    # Entities under same parent
    if same(parent1, parent2):
        tag = parent1['tag'].lower()[0]
        if tag not in ['v', 'n']:
            return (0, "null")
        if tag == 'v':
            return (1, "advise")
    
    # Entity 1 under lemma
    if parent_lemma_belongs(parent1, ['response', 'diminish', 'enhance']):
        return (1, "effect")
    elif parent_lemma_belongs(parent1, ['absorption', 'metabolism', 'presence']):
        return (1, "mechanism")
    elif parent_lemma_belongs(parent1, ['interact', 'interaction']):
        return (1, "int")
    elif parent_lemma_belongs(parent1, ['take', 'adjustment', 'avoid', 'recommend', 'contraindicate']):
        return (1, "advise")

    # Entity 2 under lemma
    if parent_lemma_belongs(parent2, ['effect']):
        return (1, "effect")
    elif parent_lemma_belongs(parent2, ['absorption', 'metabolism', 'level', 'clearance']):
        return (1, "mechanism")
    elif parent_lemma_belongs(parent2, ['take', 'caution']):
        return (1, "advise")
        
    return (0, "null")

#### Brief description of auxiliary functions

The *check_interaction* function uses some external functions:
- **get_entity_nodes:** returns the corresponding node and tokens of each entity.
- **get_entity_parent:** for each entity it returns the parent anf if it has several tokens it returns an arbitrary parent.
- **is_under:** given two entities it returns True if one entity is under the other.
- **same:** given two nodes it returns True if the both have the addres. 
- **parent_lemma_belongs:** given a lemma and a lemma set it returns true it the lemma belong to the set.

In [2]:
def get_entity_nodes(tree, entities, e1, e2):
    entity1 = []
    entity2 = []

    starts1 = [offs[0] for offs in entities[e1]]
    starts2 = [offs[0] for offs in entities[e2]]
    ends1 = [offs[1] for offs in entities[e1]]
    ends2 = [offs[1] for offs in entities[e2]]
    
    for k in sorted(tree.keys()):
        if 'start' in tree[k].keys():
            for i in range(len(starts1)):
                if int(starts1[i]) in range(tree[k]['start'], tree[k]['end']+1) or int(ends1[i]) in range(tree[k]['start'], tree[k]['end']+1):
                    entity1.append(tree[k])
                elif tree[k]['start'] in range(int(starts1[i]), int(ends1[i])+1) and tree[k]['end'] in range(int(starts1[i]), int(ends1[i])+1):
                    entity1.append(tree[k])
                    
            for i in range(len(starts2)):
                if int(starts2[i]) in range(tree[k]['start'], tree[k]['end']+1) or int(ends2[i]) in range(tree[k]['start'], tree[k]['end']+1):
                    entity2.append(tree[k])
                elif tree[k]['start'] in range(int(starts2[i]), int(ends2[i])+1) and tree[k]['end'] in range(int(starts2[i]), int(ends2[i])+1):
                    entity2.append(tree[k])
                    
    return entity1, entity2

In [3]:
def get_entity_parent(tree, entity):
    if len(entity) == 1:
        return tree[entity[0]['head']], entity[0]['rel']
    else:
        parent = None
        rel = None
        for e in entity:
            if e['head'] not in [other['address'] for other in entity]:
                parent = tree[e['head']]
                rel = e['rel']
        return parent, rel

In [4]:
def is_under(entity1, entity2):
    for i in range(len(entity1)):
        if entity1[i]['head'] in [e['address'] for e in entity2]:
            return True
    return False

In [5]:
def same(node1, node2):
    if node1['address'] == node2['address']:
        return True
    else:
        return False

In [6]:
def parent_lemma_belongs(parent, lemma_set):
    if parent['lemma'] in lemma_set:
        return True
    else:
        return False

### Results

The final results obtained for devel y test are shown below. 

#### Devel
```
SCORES FOR THE GROUP: develGoal RUN=1
Gold Dataset: /Devel

Partial Evaluation: only detection of DDI (regadless to the type)
tp      fp      fn      total   prec    recall  F1
136     250     348     484     0,3523  0,281   0,3126


Detection and classification of DDI
tp      fp      fn      total   prec    recall  F1
123     263     361     484     0,3187  0,2541  0,2828


________________________________________________________________________

SCORES FOR DDI TYPE
Scores for ddi with type mechanism
tp      fp      fn      total   prec    recall  F1
60      135     141     201     0,3077  0,2985  0,303


Scores for ddi with type effect
tp      fp      fn      total   prec    recall  F1
44      57      118     162     0,4356  0,2716  0,3346


Scores for ddi with type advise
tp      fp      fn      total   prec    recall  F1
17      38      102     119     0,3091  0,1429  0,1954


Scores for ddi with type int
tp      fp      fn      total   prec    recall  F1
2       33      0       2       0,0571  1       0,1081


MACRO-AVERAGE MEASURES FOR DDI CLASSIFICATION:
        P       R       F1
        0,2774  0,4282  0,3367
________________________________________________________________________
```
#### Test
```
SCORES FOR THE GROUP: testGoal RUN=1
Gold Dataset: /Test-DDI

Partial Evaluation: only detection of DDI (regadless to the type)
tp      fp      fn      total   prec    recall  F1
262     495     717     979     0,3461  0,2676  0,3018


Detection and classification of DDI
tp      fp      fn      total   prec    recall  F1
215     542     764     979     0,284   0,2196  0,2477


________________________________________________________________________

SCORES FOR DDI TYPE
Scores for ddi with type mechanism
tp      fp      fn      total   prec    recall  F1
88      192     214     302     0,3143  0,2914  0,3024


Scores for ddi with type effect
tp      fp      fn      total   prec    recall  F1
49      54      311     360     0,4757  0,1361  0,2117


Scores for ddi with type advise
tp      fp      fn      total   prec    recall  F1
32      137     189     221     0,1893  0,1448  0,1641


Scores for ddi with type int
tp      fp      fn      total   prec    recall  F1
46      159     50      96      0,2244  0,4792  0,3056


MACRO-AVERAGE MEASURES FOR DDI CLASSIFICATION:
        P       R       F1
        0,3009  0,2629  0,2806
________________________________________________________________________
```

### Conclusions

The final results obtained are:



## Goal 2: DDI using machine learning
### Introduction

The rule-based system implemented clearly has a lot of limitations. It is unable to reach a quite good accuracy in both Devel and Test datasets. For this reason, in this task we will use a machine learning approach.

#### ML algorithm
Specifically we use a Maximum Entropy model.

**TO-DO**
JUSTIFICATION

This time the goal is to achieve an overall F1 score of at least 0.6 on Devel dataset.

### Features
#### Used features

#### Discarded features
- Using the Dependency Tree, we also tested features with information about the distance of each entitiy to the common head between both entities (distance between the common parent and entity1, distance between common parent and entity2, and the sum of both).
- Words and POS tags of tokens before, in between and after the entities (in the same way to what we finally do with lemmas).
- Feature indicating if the parent lemma of one entity belongs to a set of lemmas, one for each type of drug-drug interaction. In fact, these sets of lemmas used were the ones extracted from the data exploration and used also in the rule-based system. After testing, we end up with only maintaining it for the interaction type.
- We also tested a feature indicating if both entities were under same head and this was a noun.

In [1]:
def extract_features(tree, entities, e1, e2):
    # Get entities
    entity1, entity2 = get_entity_nodes(tree, entities, e1, e2)
    
    # Get head and rel of each entity
    parent1, rel1 = get_entity_parent(tree, entity1)
    parent2, rel2 = get_entity_parent(tree, entity2)
    
    # Features
    features = ['h1_lemma=%s' % parent1['lemma'],
                'h1_word=%s' % parent1['word'],
                'h1_tag=%s' % parent1['tag'],
                'h1_rel=%s' % rel1,
                'h2_lemma=%s' % parent2['lemma'],
                'h2_word=%s' % parent2['word'],
                'h2_tag=%s' % parent2['tag'],
                'h2_rel=%s' % rel2,
                ]
    
    if same(parent1, parent2):
        features.append('under_same')
        if parent1['tag'][0].lower() == 'v':
            features.append('under_same_verb')
    
    if is_under(entity1, entity2):
        features.append('1under2')
    
    if is_under(entity2, entity1):
        features.append('2under1')
        
    if parent_lemma_belongs(parent1, ['interact', 'interaction']):
        features.append('interaction')
    
    start1, end1 = get_entity_limits(entity1)
    start2, end2 = get_entity_limits(entity2)
    for k in sorted(tree.keys()):
        if 'start' in tree[k].keys() and  tree[k]['start'] < start1:
            features.append('lb1=%s' % tree[k]['lemma'])

        if 'start' in tree[k].keys() and end1 < tree[k]['start'] < start2:
            features.append('lib=%s' % tree[k]['lemma'])

        if 'start' in tree[k].keys() and end2 < tree[k]['start']:
            features.append('la2=%s' % tree[k]['lemma'])
      
    return features

In [ ]:
def get_entity_limits(entity):
    start = min([e['start'] for e in entity])
    end = max([e['end'] for e in entity])
    return start, end

### Learner

In [ ]:
def train(megam, parameters, features_file, out_train_model):
    print("ubuntu run \""+ megam + " " + parameters + " multiclass " + features_file + " > " + out_train_model + "\"")
    os.system("ubuntu run \""+ megam + " " + parameters + " multiclass " + features_file + " > " + out_train_model + "\"")### Learner

In [ ]:
parameters = '-quiet -nc -repeat 5 -tune -lambda 0.01 -minfc 3'
train(model_path+'megam.opt', parameters, features_path+'train_features', model_path+'model.dat')

### Classifier

In [ ]:
def classify(megam, parameters, features_file, train_model, prediction_output):
    print("ubuntu run \""+ megam + " " + parameters + " -predict " + train_model + " multiclass " + features_file + " > " + prediction_output + "\"")
    os.system("ubuntu run \""+ megam + " " + parameters + " -predict " + train_model + " multiclass " + features_file + " > " + prediction_output + "\"")

In [ ]:
# Devel prediction
classify(model_path+'megam.opt', '-quiet -nc ', features_path+'devel_features', model_path+'model.dat', output_path+'devel_prediction')

In [ ]:
# Test prediction
classify(model_path+'megam.opt', '-quiet -nc ', features_path+'test_features', model_path+'model.dat', output_path+'test_prediction')

### Results

#### Devel
```
SCORES FOR the file: task9.2_develGoal.txt
Gold Dataset: /Devel

Partial Evaluation: only detection of DDI (regadless to the type)
tp	fp	fn	total	prec	recall	F1
260	99	224	484	0,7242	0,5372	0,6168


Detection and Classification of DDI
tp	fp	fn	total	prec	recall	F1
240	119	244	484	0,6685	0,4959	0,5694


________________________________________________________________________

SCORES FOR DDI TYPE
Scores for ddi with type mechanism
tp	fp	fn	total	prec	recall	F1
86	65	115	201	0,5695	0,4279	0,4886


Scores for ddi with type effect
tp	fp	fn	total	prec	recall	F1
99	37	63	162	0,7279	0,6111	0,6644


Scores for ddi with type advise
tp	fp	fn	total	prec	recall	F1
53	16	66	119	0,7681	0,4454	0,5638


Scores for ddi with type int
tp	fp	fn	total	prec	recall	F1
2	1	0	2	0,6667	1	0,8


MACRO-AVERAGE MEASURES:
	P	R	F1
	0,6831	0,6211	0,6506
________________________________________________________________________
```

#### Test
```
SCORES FOR the file: task9.2_testGoal.txt
Gold Dataset: /Test-DDI

Partial Evaluation: only detection of DDI (regadless to the type)
tp	fp	fn	total	prec	recall	F1
714	521	265	979	0,5781	0,7293	0,645


Detection and Classification of DDI
tp	fp	fn	total	prec	recall	F1
619	616	360	979	0,5012	0,6323	0,5592


________________________________________________________________________

SCORES FOR DDI TYPE
Scores for ddi with type mechanism
tp	fp	fn	total	prec	recall	F1
214	213	88	302	0,5012	0,7086	0,5871


Scores for ddi with type effect
tp	fp	fn	total	prec	recall	F1
222	221	138	360	0,5011	0,6167	0,5529


Scores for ddi with type advise
tp	fp	fn	total	prec	recall	F1
146	156	75	221	0,4834	0,6606	0,5583


Scores for ddi with type int
tp	fp	fn	total	prec	recall	F1
37	26	59	96	0,5873	0,3854	0,4654


MACRO-AVERAGE MEASURES:
	P	R	F1
	0,5183	0,5928	0,553
________________________________________________________________________
```


### Conclusions

The final results obtained are: